## 4.0 Импорты библиотек

In [9]:
import os
import yaml
import logging
import numpy as np
import scipy.stats as stats
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import levene
from typing import List, Any, Optional, Tuple, Dict
from datetime import datetime
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr, kurtosis, skew
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.base import BaseEstimator, TransformerMixin

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
# расширяем поле ноутбука для удобства
from IPython.display import display, HTML
display(HTML('<style>.container {width:87% !important;}</style>'))
display(HTML("<style>.output_scroll {height:auto !important; max-height:10000px !important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [12]:
# Настройки для pandas (количество отображаемых колонок)
pd.set_option('display.max_columns', 100)

In [13]:
# Определение стиля для pyplot
plt.style.use('ggplot')

In [14]:
# Текущая рабочая директория
cwd = Path().resolve()

# Поднимаемся на один уровень выше
project_root = cwd.parent

# Добавляем корень проекта в sys.path
sys.path.append(str(project_root))

# Загрузка данных из config.yaml
from src.data import downloader, loader, preprocessor, saving
from src.features import feat_preprocessing

# Путь к файлу config.yaml
config_path = project_root / "config" / "config.yaml"

# Загружаем конфиг
config = loader.load_config(config_path)

## 4.1. Загрузка данных

In [15]:
# Загрузка train
df_train = loader.data_load_preprocessed(data_type='train', config=config)

D:\Skills\Kaggle\ml-regression_concrete-strength
D:\Skills\Kaggle\ml-regression_concrete-strength\data\processed\eda_data_train.pkl
[⧗] Загружаю данные из: D:\Skills\Kaggle\ml-regression_concrete-strength\data\processed\eda_data_train.pkl
[✓] Данные успешно загружены. Форма: (781, 11)


In [16]:
# Вывод первых 5 строк тренировочного датасета
df_train.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength,W/C,Sp/C_pct
0,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,16.28,0.570745,0.000000
1,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,59.59,0.427699,0.007943
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,13.82,0.749600,0.022000
3,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,35.76,0.619355,0.000000
4,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,33.40,0.768743,0.032923


In [17]:
# Загрузка test
df_test = loader.data_load_preprocessed(data_type='test', config=config)

D:\Skills\Kaggle\ml-regression_concrete-strength
D:\Skills\Kaggle\ml-regression_concrete-strength\data\processed\eda_data_test.pkl
[⧗] Загружаю данные из: D:\Skills\Kaggle\ml-regression_concrete-strength\data\processed\eda_data_test.pkl
[✓] Данные успешно загружены. Форма: (228, 10)


In [18]:
# Вывод первых 5 строк тестового датасета
df_test.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,W/C,Sp/C_pct
0,167.4,129.9,128.6,175.5,7.8,1006.3,746.6,28,1.048387,0.046595
1,475.0,118.8,0.0,181.1,8.9,852.1,781.5,7,0.381263,0.018737
2,251.4,0.0,118.3,188.5,6.4,1028.4,757.7,100,0.749801,0.025457
3,307.0,0.0,0.0,193.0,0.0,968.0,812.0,365,0.628664,0.000000
4,143.6,0.0,174.9,158.4,17.9,942.7,844.5,28,1.103064,0.124652


## 4.2. Предобработка данных

In [ ]:
# Разделение на признаки и целевую переменную
X = df_train.drop('Strength', axis=1)
y = df_train['Strength']
y_name = y.name

In [ ]:
# Все доступные стратегии
all_strategies = [
    ['abnormal'],
    ['combine'],
    ['gost_binar'],
    ['gost_remove'],
    ['iqr_remove'],
    ['gost_binar', 'combine']
]

results = []

# Тит используемых алгоритмов
model_type = 'trees_models'

# Определяем приоритетный список для доменных знаний о бетоне
DOMAIN_PRIORITY_LIST = [
    # 'Water',              # Самый важный - критически влияет на прочность
    'W/C',
    'Cement',             # Второй по важности - основной вяжущий компонент
    'Age',                # Время твердения - важный технологический параметр
    'Superplasticizer',   # Химическая добавка
    'Fly Ash',            # Минеральная добавка
    'Blast Furnace Slag', # Минеральная добавка  
    'Coarse Aggregate',   # Крупный заполнитель
    'Fine Aggregate'     # Мелкий заполнитель - наименее важный
    'Has_Slag',           # Наличие шлака в смеси (0/1)
    'Has_FlyAsh',         # Наличие золы в смеси (0/1)
    'Has_Superplasticizer', # Наличие суперпластификатора (0/1)
    'Low_WC_ratio',       # Низкое В/Ц отношение (адаптивный порог)
    'High_WC_ratio',      # Высокое В/Ц отношение (адаптивный порог)
    'Low_WC_tech',        # Низкое В/Ц отношение (< 0.4)
    'High_WC_tech'       # Высокое В/Ц отношение (> 0.6)
]

# Конфигурация для преобразования признаков
# Сделай так:
feature_config = {
    'trend_settings': {
        'names': ['Linear', 'Log', 'Sqrt']  # Только безопасные
    }
}

for i, strategies in enumerate(all_strategies, 1):
    print(f"\n{'='*75}")
    print(f"СТРАТЕГИЯ {i}: {strategies}")
    print(f"{'='*75}")
    
    # Создание обработчика для текущей стратегии    
    outlier_handler = feat_preprocessing.OutlierHandler(
        strategies=strategies,
        config=config,
        target_col=y_name
    )
   
    try:
        # 1. Обработка выбросов
        X_processed, y_processed = outlier_handler.fit_transform(X, y)
        summary  = outlier_handler.get_outlier_summary()

        # Отчет по обработки выбросов
        print(f"\n{'-'*75}")
        print("ОТЧЕТ ПО ОБРАБОТКИ ВЫБРОСОВ")
        print(f"{'-'*75}")        
        print("\nДетали по выбросам:")
        for feature, outliers in summary['outliers_by_feature'].items():
            for outlier_info in outliers:
                print(f"  - {feature}: {outlier_info['count']} (порог: {outlier_info['threshold']})")

        summary  = outlier_handler.get_outlier_summary()
                

        # 2. Feature Engineering        
        feature_engineer = feat_preprocessing.FeatureHandleEngineering(model_type=model_type,
                                                                       wc_column='W/C')
        
        X_with_features = feature_engineer.fit_transform(X_processed)
        feature_summary = feature_engineer.get_feature_summary()

        # Вывод информации о feature engineering
        print(f"\n{'-'*75}")
        print("ОТЧЕТ ПО СОЗДАНИЮ ИНЖЕНЕРНЫХ ПРИЗНАКОВ")
        print(f"{'-'*75}") 
        print(f"\nСозданные инженерные признаки:")
        feature_descriptions = feature_engineer.get_feature_descriptions()
        for feature, description in feature_descriptions.items():
            print(f"  - {feature}: {description}")
            
                
        # 3. Преобразование признаков        
        print(f"\n{'-'*75}")
        print("ОТЧЕТ О ПРЕОБРАЗОВАНИИ ПРИЗНАКОВ")
        print(f"{'-'*75}")
        
        feature_transformer = feat_preprocessing.FeatureTransformer(
            config=feature_config,
            target_col=y_name
        )
        X_transformed = feature_transformer.fit_transform(X_with_features, y_processed)

        # Подсчет статистики по преобразованиям
        transformation_report = feature_transformer.get_transformation_report()
        transform_stats = {}
        for _, row in transformation_report.iterrows():
            transform_type = row['Лучшее преобразование']
            transform_stats[transform_type] = transform_stats.get(transform_type, 0) + 1
       
        # Вывод информации о преобразованных признаках
        print()        
        feature_transformer.get_transformation_report()

        print()        
        print("Распределение преобразований:")
        for transform_type, count in transform_stats.items():
            print(f"  - {transform_type}: {count} признаков")       

        
        # 4. Анализ бинарных признаков
        binary_analyzer = feat_preprocessing.ZeroBinaryEncoder(alpha=0.05)
        X_with_binary = binary_analyzer.fit_transform(X_transformed, y_processed)
        binary_summary = binary_analyzer.get_summary()

        # Вывод отчета по бинарным признакам
        print("\n" + "-"*75)
        print("АНАЛИЗ ЗНАЧИМОСТИ НУЛЕВЫХ ЗНАЧНИЙ ДЛЯ ПРИЗНАКОВ:")
        print("-"*75)
        binary_analyzer.print_detailed_report()
      
        
        # 5. Удаление неинформативных признаков         
        uninform_remover = feat_preprocessing.FeatureUninformRemove(threshold=0.95,
                                                                    verbose=True)
        
        X_after_uninform = uninform_remover.fit_transform(X_with_binary)
        uninform_summary = uninform_remover.get_removal_summary()

        
        # 6. Анализ мультиколлинеарности
        print(f"\n{'-'*75}")
        print("АНАЛИЗ МУЛЬТИКОЛЛИНЕАРНОСТИ")
        print(f"{'-'*75}")
        
        multicollinearity_handler = feat_preprocessing.CollinearityReducer(
            vif_threshold=15,
            correlation_threshold=0.90,
            priority_strategy='domain_priority',
            domain_priority_list=DOMAIN_PRIORITY_LIST,
            verbose=True
        )
        
        X_final = multicollinearity_handler.fit_transform(X_after_uninform)
        multicollinearity_report = multicollinearity_handler.get_removal_report()       


        # Дополнительная визуализация мультиколлинеарности
        if multicollinearity_report['removed_features']:
            print(f"\n{'-'*40}")
            print("Детальный анализ мультиколлинеарности")
            print(f"{'-'*40}")
            
            # Показываем VIF до и после обработки
            initial_vif = multicollinearity_report['vif_analysis']['initial_vif']
            final_vif = multicollinearity_report['vif_analysis']['final_vif']
            
            print(f"VIF анализ:")
            print(f"  - Итераций выполнено: {multicollinearity_report['vif_analysis']['iterations']}")
            print(f"  - Максимальный VIF до обработки: {max(initial_vif.values()):.2f}")
            print(f"  - Максимальный VIF после обработки: {max(final_vif.values()):.2f}")
            
            # Показываем экстремальные корреляции
            if multicollinearity_report['correlation_analysis'].get('extreme_pairs'):
                print(f"Обнаружено экстремальных корреляций: {len(multicollinearity_report['correlation_analysis']['extreme_pairs'])}")  
        
        
        # Сохранение результатов
        result = {
            'strategies': strategies,
            'X_original': X.shape[0],
            'X_processed': X_final.shape[0],
            'X_original_shape': X.shape,
            'X_processed_shape': X_final.shape,
            'rows_removed': summary['removed_rows'],
            'removal_percent': (summary['removed_rows'] / X.shape[0]) * 100,
            'features_created': feature_summary['total_features_created'],
            'binary_features': len(binary_summary['binary_features_created']),
            'transformed_features': len(transformation_report),  # Количество преобразованных признаков
            'uninform_features_removed': uninform_summary['total_removed'],
            'multicollinearity_features_removed': len(multicollinearity_report['removed_features']),
            'outlier_handler': outlier_handler,
            'feature_engineer': feature_engineer,
            'feature_transformer': feature_transformer,  # Сохраняем трансформер
            'binary_analyzer': binary_analyzer,
            'uninform_remover': uninform_remover,
            'multicollinearity_handler': multicollinearity_handler,
            'X_processed_data': X_final,
            'y_processed_data': y_processed
        }
        results.append(result)
        
        # Вывод результатов
        print(f"\n{'-'*75}")
        print("ИТОГИ ОБРАБОТКИ ДАННЫХ")
        print(f"{'-'*75}")
        print(f"Обработано стратегиями: {', '.join(summary['strategies_applied'])}")
        print(f"Исходный размер: {X.shape}")
        print(f"После обработки выбросов: {X_processed.shape}")
        print(f"После feature engineering: {X_with_features.shape}")        
        print(f"После преобразования признаков: {X_transformed.shape}")
        print(f"После анализа бинарных признаков: {X_with_binary.shape}")
        print(f"После удаления неинформативных: {X_after_uninform.shape}")
        print(f"После удаления мультиколлинеарных: {X_final.shape}")        
        print(f"Удалено строк: {summary['removed_rows']} ({result['removal_percent']:.1f}%)")
        print(f"Создано инженерных признаков: {feature_summary['total_features_created']}")
        print(f"Создано бинарных признаков: {len(binary_summary['binary_features_created'])}")
        print(f"Преобразовано признаков: {len(transformation_report)}")        
        print(f"Удалено неинформативных признаков: {uninform_summary['total_removed']}")
        print(f"Удалено мультиколлинеарных признаков: {len(multicollinearity_report['removed_features'])}")
        print(f"Целевая переменная: {y_processed.shape}")      

        if summary['binary_features_created']:
            print(f"Бинарные признаки от выбросов: {', '.join(summary['binary_features_created'])}")
            
        if binary_summary['binary_features_created']:
            print(f"Бинарные признаки от значимости: {', '.join(binary_summary['binary_features_created'])}")
        
        if uninform_summary['columns_removed']:
            print(f"Удаленные неинформативные признаки: {', '.join(uninform_summary['columns_removed'])}")

        if multicollinearity_report['removed_features']:
            print(f"Удаленные мультиколлинеарные признаки: {', '.join(multicollinearity_report['removed_features'])}")
            print(f"Стратегия удаления: {multicollinearity_handler.priority_strategy}")
       

        
        # ⭐ Вывод финальной информации о признаках
        print(f"\n{'-'*75}")
        print("ФИНАЛЬНАЯ СТРУКТУРА ДАННЫХ")
        print(f"{'-'*75}")
        print(f"Всего признаков: {X_final.shape[1]}")
        print(f"Типы признаков:")
        if isinstance(X_final, pd.DataFrame):
            print(X_final.dtypes.value_counts())
        print(f"Размерность: {X_final.shape[0]} строк × {X_final.shape[1]} признаков")
        
  
       
                
    except Exception as e:
        print(f"Ошибка при обработке стратегией {strategies}: {e}")
        import traceback
        traceback.print_exc()
        results.append({
            'strategies': strategies,
            'error': str(e),
            'X_processed': X.shape[0],
            'X_processed_shape': X.shape,
            'rows_removed': 0,
            'features_created': 0,
            'transformed_features': 0,
            'binary_features': 0,
            'uninform_features_removed': 0,
            'multicollinearity_features_removed': 0
        })

In [ ]:
# Создаем сводную таблицу с информацией о feature engineering
summary_data = []
for i, result in enumerate(results, 1):
    if 'error' not in result:
        summary_data.append({
            'Стратегия': f"{i}: {result['strategies']}",
            'Исходный размер': f"{result['X_original_shape'][0]}×{result['X_original_shape'][1]}",
            'Итоговый размер': f"{result['X_processed_shape'][0]}×{result['X_processed_shape'][1]}",
            'Удалено строк': result['rows_removed'],
            'Процент удаления': f"{result['removal_percent']:.1f}%",
            'Инженерные признаки': result['features_created'],
            '⭐ Преобразовано признаков': result['transformed_features'],
            'Бинарные признаки': result['binary_features'],
            '⭐ Неинформ. удалено': result['uninform_features_removed'],
            '⭐ Мультиколл. удалено': result['multicollinearity_features_removed'],
            'Всего признаков': result['X_processed_shape'][1],
            'Эффективность обработки': f"{(result['X_processed_shape'][0] / result['X_original_shape'][0] * 100):.1f}%"})
    else:
        summary_data.append({
            'Стратегия': f"{i}: {result['strategies']}",
            'Исходный размер': f"{X.shape[0]}×{X.shape[1]}",
            'Итоговый размер': "ОШИБКА",
            'Удалено строк': 0,
            'Процент удаления': "0%",
            'Инженерные признаки': 0,
            '⭐ Преобразовано признаков': 0,
            'Бинарные признаки': 0,
            '⭐ Неинформ. удалено': 0,
            '⭐ Мультиколл. удалено': 0,
            'Всего признаков': "ОШИБКА",
            'Эффективность обработки': "0%"
        })

# Создаем DataFrame для красивого отображения
summary_df = pd.DataFrame(summary_data)
summary_df

In [ ]:
X_final

In [ ]:
X_final['Outlier_cement'].value_counts()